# Quran: Load Quran text (Arabic and English) into a file via public APIs

In [27]:
import requests
import pandas as pd

def getSurah(x: int) -> dict:
    """Get Surah Arabic and English as dictionary"""
    df = []
    url = f"http://api.alquran.cloud/v1/surah/{x}"
    response = requests.get(url)
    translation = requests.get(url + "/en.asad")
    response.raise_for_status()
    translation.raise_for_status()
    ordered = response.json()
    trans = translation.json()
    data = ordered["data"]["ayahs"]
    tran = trans["data"]["ayahs"]
    if len(data) != len(tran):
        print("Error!")
    else:
        n = len(data)
        for i in range(n):
            df.append({
                "Arabic": (data[i]["text"]),
                "English": tran[i]["text"]
            })
    return df
def surahToDB(df: dict):
    db = pd.DataFrame(df)
    return db
def saveSurah(x: int):
    """Save Surah to local file as dataframe"""
    df = getSurah(x)
    pass

In [26]:
from PIL import Image, ImageDraw, ImageFont
import textwrap
import arabic_reshaper
from bidi.algorithm import get_display

def create_text_image(df, arabic_column, english_column, image_path='output_image.png'):
    # Load a font
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()

    # Define text wrapping parameters
    wrapper = textwrap.TextWrapper(width=35, expand_tabs=False, replace_whitespace=False)

    # Calculate total height required for all text
    total_height = 0
    for index, row in df.iterrows():
        arabic_lines = wrapper.wrap(text=row[arabic_column])
        english_lines = wrapper.wrap(text=row[english_column])
        total_height += (len(arabic_lines) + len(english_lines)) * 25 + 10  # Line height + spacing

    # Set image dimensions
    img_width = 800
    img_height = total_height + 50  # Additional padding
    img = Image.new('RGB', (img_width, img_height), color='white')
    draw = ImageDraw.Draw(img)

    # Starting position for text
    y_position = 10

    for index, row in df.iterrows():
        # Wrap Arabic and English texts
        arabic_lines = wrapper.wrap(text=row[arabic_column])
        english_lines = wrapper.wrap(text=row[english_column])

        # Reshape and reorder Arabic text
        reshaped_arabic = arabic_reshaper.reshape(row[arabic_column])
        bidi_arabic = get_display(reshaped_arabic)

        # Now wrap the processed Arabic text instead of the original
        arabic_lines = wrapper.wrap(text=bidi_arabic)
        english_lines = wrapper.wrap(text=row[english_column])
        
        # Draw Arabic text
        for line in arabic_lines:
            draw.text((10, y_position), line, font=font, fill='black')
            y_position += 25

        # Draw English text
        for line in english_lines:
            draw.text((img_width // 2, y_position), line, font=font, fill='black')
            y_position += 25

        # Add spacing between rows
        y_position += 10

    # Save the image
    img.save(image_path)
    img.show()

In [28]:
df1 = getSurah(1)
print(df1)
df2 = surahToDB(df1)
create_text_image(df2, "Arabic", "English", image_path='output_image.png')

[{'Arabic': 'بِسۡمِ ٱللَّهِ ٱلرَّحۡمَـٰنِ ٱلرَّحِیمِ\n', 'English': 'In the name of God, The Most Gracious, The Dispenser of Grace:'}, {'Arabic': 'ٱلۡحَمۡدُ لِلَّهِ رَبِّ ٱلۡعَـٰلَمِینَ\n', 'English': 'All praise is due to God alone, the Sustainer of all the worlds,'}, {'Arabic': 'ٱلرَّحۡمَـٰنِ ٱلرَّحِیمِ\n', 'English': 'The Most Gracious, the Dispenser of Grace,'}, {'Arabic': 'مَـٰلِكِ یَوۡمِ ٱلدِّینِ\n', 'English': 'Lord of the Day of Judgment!'}, {'Arabic': 'إِیَّاكَ نَعۡبُدُ وَإِیَّاكَ نَسۡتَعِینُ\n', 'English': 'Thee alone do we worship; and unto Thee alone do we turn for aid.'}, {'Arabic': 'ٱهۡدِنَا ٱلصِّرَ\u200aٰ\u2060طَ ٱلۡمُسۡتَقِیمَ\n', 'English': 'Guide us the straight way.'}, {'Arabic': 'صِرَ\u200aٰ\u2060طَ ٱلَّذِینَ أَنۡعَمۡتَ عَلَیۡهِمۡ غَیۡرِ ٱلۡمَغۡضُوبِ عَلَیۡهِمۡ وَلَا ٱلضَّاۤلِّینَ\n', 'English': 'The way of those upon whom Thou hast bestowed Thy blessings, not of those who have been condemned [by Thee], nor of those who go astray!'}]


In [21]:
getSurah(1)

[{'Arabic': 'ﻢﯿﺣﺮﻟﭐ ﻦـﻤﺣﺮﻟﭐ ﻪﻠﻟﭐ ﻢﺴﺑ\n',
  'English': 'In the name of God, The Most Gracious, The Dispenser of Grace:'},
 {'Arabic': 'ﻦﯿﻤﻠـﻌﻟﭐ ﺏﺭ ﻪﻠﻟ ﺪﻤﺤﻟﭐ\n',
  'English': 'All praise is due to God alone, the Sustainer of all the worlds,'},
 {'Arabic': 'ﻢﯿﺣﺮﻟﭐ ﻦـﻤﺣﺮﻟﭐ\n',
  'English': 'The Most Gracious, the Dispenser of Grace,'},
 {'Arabic': 'ﻦﯾﺪﻟﭐ ﻡﻮﯾ ﻚﻠـﻣ\n', 'English': 'Lord of the Day of Judgment!'},
 {'Arabic': 'ﻦﯿﻌﺘﺴﻧ ﻙﺎﯾﺇﻭ ﺪﺒﻌﻧ ﻙﺎﯾﺇ\n',
  'English': 'Thee alone do we worship; and unto Thee alone do we turn for aid.'},
 {'Arabic': 'ﻢﯿﻘﺘﺴﻤﻟﭐ ﻁ\u200aﺮﺼﻟﭐ ﺎﻧﺪﻫﭐ\n',
  'English': 'Guide us the straight way.'},
 {'Arabic': 'ﻦﯿﻟﺎﻀﻟﭐ ﻻﻭ ﻢﻬﯿﻠﻋ ﺏﻮﻀﻐﻤﻟﭐ ﺮﯿﻏ ﻢﻬﯿﻠﻋ ﺖﻤﻌﻧﺃ ﻦﯾﺬﻟﭐ ﻁ\u200aﺮﺻ\n',
  'English': 'The way of those upon whom Thou hast bestowed Thy blessings, not of those who have been condemned [by Thee], nor of those who go astray!'}]